In [10]:
import pandas as pd
import geopandas
import geoplot
import re
import osmnx as ox
import matplotlib.pyplot as plt
import numpy as np

In [2]:
place_name = "Shorewood, Wisconsin, USA"
graph = ox.graph_from_place(place_name)
area = ox.gdf_from_place(place_name)

/home/imbrius/anaconda3/envs/geo_env/lib/python3.8/site-packages/osmnx/boundaries.py:36: UserWarning: The `boundaries` module has been deprecated and will be removed in a future relase. Use the `geocoder` module's `geocode_to_gdf` function instead.
  warnings.warn(msg)


**Cells above this line should only be run once as they hit public web services.**

In [3]:
def plot_map():
    nodes, edges = ox.graph_to_gdfs(graph)
    fix, ax = plt.subplots()
    area.plot(ax=ax, facecolor="black")
    edges.plot(ax=ax, linewidth=1, edgecolor="#BC8F8F")

In [4]:
def sort_stops(stops):
    stops_block = []
    stops_intersection = []
    
    for i in stops.index:
        stop = stops.loc[i, 'Location']
        
        if stop.find('/') == -1:
            stops_block.append(stops.loc[i])
        else:
            stops_intersection.append(stops.loc[i])
    
    return pd.DataFrame(stops_block), pd.DataFrame(stops_intersection)

def prepare_lcn_data(lcn_data):
    stopwords = ['BLK', ',LOT', ',PARK', ',BLDG']
    
    for i in lcn_data.index:
        addr = lcn_data.loc[i, 'Location']
        addr = re.sub(';[A-Z][A-Z]', '', addr)
        
        for word in stopwords:
            addr = addr.replace(word, '')
            
        lcn_data.loc[i, 'Location'] = addr
    
    return sort_stops(lcn_data)

In [5]:
traffic_stops = geopandas.read_file("Copy of Sept 2019 SHPD Traffic Stops.csv")

# For now, we are ignoring the stops_intersection list
# as none of the free geocoders support intersection lookups.
# Google Maps API v3 does but it's a paid subscription service.
# Watch the OSM Nominatim project
# (https://github.com/osm-search/Nominatim/issues/123)
# for when this gets implemented.

stops_block, stops_intersection = prepare_lcn_data(traffic_stops)
lcns = []

for i in stops_block.index:
    print(f"Geocoding address {stops_block.loc[i, 'Location']}")
    
    try:
        lcn = {}
        lcn['Call#'] = stops_block.loc[i, 'Call#']
        lcn['geometry'] = ox.geocoder.geocode(stops_block.loc[i, 'Location'])
    except Exception as ex:
        print(f"Error: {ex}");
    finally:
        lcns.append(lcn)

Geocoding address 1700 E Capitol Dr, Shorewood, WI 53211
Geocoding address 3800 N Maryland Ave, Shorewood, WI 53211
Geocoding address 3300 N Oakland Ave, WI 53211
Geocoding address 4000 N Wilson Dr, Shorewood, WI 53211
Geocoding address 3900 N Prospect Ave, Shorewood, WI 53211
Geocoding address 1900 E Capitol Dr, Shorewood, WI 53211
Geocoding address 3800 N Maryland Ave, Shorewood, WI 53211
Geocoding address 3500 N Oakland Ave, Shorewood, WI 53211
Geocoding address 3800 N Morris Blvd, Shorewood, WI 53211
Geocoding address 3500 N Oakland Ave, Shorewood, WI 53211
Geocoding address 3800 N Maryland Ave, Shorewood, WI 53211
Geocoding address 4000 N Humboldt Ave, Milwaukee, WI 53211
Geocoding address 3600 N Oakland Ave, Shorewood, WI 53211
Geocoding address 1325 E Capitol Dr, Shorewood, WI 53211
Geocoding address 2000 E Capitol Dr, Shorewood, WI 53211
Geocoding address 3900 N Morris Blvd, Shorewood, WI 53211
Geocoding address 3500 N Oakland Ave, Shorewood, WI 53211
Geocoding address 3800 N O

Geocoding address 1300 E Capitol Dr, Shorewood, WI 53211
Geocoding address 1600 E Capitol Dr, Shorewood, WI 53211
Geocoding address 100 E Capitol Dr, Glendale, WI 53212
Error: Nominatim geocoder returned no results for query "100 E Capitol Dr, Glendale, WI 53212"
Geocoding address 4000 N Bartlett Ave, Shorewood, WI 53211
Geocoding address 3900 N Oakland Ave, Shorewood, WI 53211
Geocoding address 3300 N Oakland Ave, Milwaukee, WI 53211
Geocoding address 1800 E Edgewood Ave, Shorewood, WI 53211
Geocoding address 2100 E Capitol Dr, Shorewood, WI 53211
Geocoding address 1500 E Capitol Dr, Shorewood, WI 53211
Geocoding address 4000 N Newhall St, Shorewood, WI 53211
Geocoding address 1000 E Capitol Dr, Milwaukee, WI 53211
Geocoding address 3700 N Oakland Ave, Shorewood, WI 53211
Geocoding address 1600 E Capitol Dr, Shorewood, WI 53211
Geocoding address 4000 N Wilson Dr, Shorewood, WI 53211
Geocoding address 1500 E Capitol Dr, Shorewood, WI 53211
Geocoding address 700 E. Capitol DR; Milwaukee

In [86]:
lcns_frame = pd.DataFrame(lcns)
geo_column = lcns_frame['geometry'].tolist()
lcns_frame[['x', 'y']] = pd.DataFrame(geo_column, columns=['x', 'y'])
lcns_gdf = geopandas.GeoDataFrame(lcns_frame, geometry=geopandas.points_from_xy(lcns_frame['x'], lcns_frame['y']))
merged_frame = stops_block.merge(lcns_gdf, on='Call#')

for col in ['geometry_x', 'x', 'y']:
    merged_frame = merged_frame.drop(col, 1)
    
merged_frame = merged_frame.rename(columns={'geometry_y': 'geometry'}).set_geometry('geometry')
merged_frame

# note - took me an entire afternoon to get the geocoded coords merged back into the GeoDataFrame
# geopandas, please make this easier

,Juris,Date,Call#,CFS,Location,Sex,Race,Age,Residence,Reason for the Stop,Disposition,geometry
0,SH,09/09/19 08:09,19.009580,TS,"1700 E Capitol Dr, Shorewood, WI 53211",F,A,33,"Shorewood, WI",No seatbelt,CIT,POINT (43.08927 -87.88862)
1,SH,09/16/19 10:23,19.009868,TS,"3800 N Maryland Ave, Shorewood, WI 53211",M,A,25,"Milwaukee, WI",Expired registration,CIT,POINT (43.08615 -87.88248)
2,SH,09/25/19 11:00,19.010273,TS,"3300 N Oakland Ave, WI 53211",M,A,43,"Shorewood, WI",Failure to yield right of way,CIT,POINT (43.07775 -87.88766)
3,SH,09/03/19 12:49,19.009324,TS,"4000 N Wilson Dr, Shorewood, WI 53211",F,B,19,"Glendale, WI",Registration check showed owner has suspended ...,CIT,POINT (43.08934 -87.89496)
4,SH,09/03/19 13:15,19.009326,TS,"3900 N Prospect Ave, Shorewood, WI 53211",F,B,22,"Milwaukee, WI",No license plate displayed,CIT,POINT (43.08717 -87.87998)
...,...,...,...,...,...,...,...,...,...,...,...,...
240,SH,09/28/19 11:58,19.010391,TS,"3500 N Oakland Ave, Shorewood, WI 53211",M,W,39,"Shorewood, WI",Illegal turn,WAR,POINT (43.08206 -87.88754)
241,SH,09/30/19 05:44,19.010427,TS,"1300 E Capitol Dr, Shorewood, WI 53211",F,W,30,"Milwaukee, WI",Operating w/o headlights,WAR,POINT (43.08929 -87.89490)
242,SH,09/30/19 06:00,19.010429,TS,"4100 N Wilson Dr, Shorewood, WI 53211",F,W,25,"Milwaukee, WI",Speeding,WAR,POINT (43.09077 -87.89579)
243,SH,09/30/19 12:36,19.010437,TS,"1100 E Capitol Dr, Shorewood, WI 53211",F,W,22,"Shorewood, WI",Expired registration,WAR,POINT (43.09055 -87.89728)
